In [ ]:
import re
import numpy as np
from collections import Counter, defaultdict
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

# Okt 형태소 분석기 초기화
okt = Okt()

def preprocess_synopsis(synopsis, genre, actors, genre_weight=1, actor_weight=1):
    """
    영화 시놉시스, 장르, 배우 정보를 전처리하는 함수.

    Args:
        synopsis: 시놉시스 텍스트.
        genre: 장르 (문자열).
        actors: 배우 리스트.
        genre_weight: 장르 가중치 (몇 번 반복할지).
        actor_weight: 배우 가중치.

    Returns:
        전처리된 토큰 리스트.
    """

    # 1. 특수문자 제거 (., ", ' 제외)
    synopsis = re.sub(r"[^가-힣A-Za-z0-9.,\"']+", " ", synopsis)

    # 2. 불용어 제거 (직접 정의)
    stopwords = ["은", "는", "이", "가", "을", "를", "에게", "의", "와", "과", "만", "도", "에", "고", "다", "것", "로", "게", "두", "들"]

    # 3. 명사 추출 (KoNLPy 사용)
    nouns = okt.nouns(synopsis)  # 명사만 추출
    filtered_nouns = [noun for noun in nouns if noun not in stopwords] # 불용어 제거

    # 4. 장르, 배우 정보 추가 (가중치 적용)
    genre_tokens = [genre.lower()] * genre_weight
    actor_tokens = [actor.lower() for actor in actors] * actor_weight

    # 명사 토큰, 장르 토큰, 배우 토큰을 모두 합칩니다.
    all_tokens = filtered_nouns + genre_tokens + actor_tokens

    return all_tokens



def create_tfidf_matrix(documents_tokens):
    """
    문서별 토큰 리스트로부터 TF-IDF 행렬을 생성하는 함수

    Args:
      documents_tokens: 문서별 토큰 리스트의 리스트 (예: [["나", "영화"], ["영화", "재미"]])

    Returns:
        TF-IDF 행렬 (NumPy 배열), 단어 사전 (dict)
    """

    # 1. DTM 및 단어 사전 생성
    def create_dtm(documents_tokens):
      # 1. 단어 사전 만들기
      word_dict = defaultdict(lambda: len(word_dict))

      # 2. DTM 생성
      dtm = []
      for tokens in documents_tokens:
          term_frequency = {}  # 문서 내 단어 빈도
          for token in tokens:
              term_frequency[word_dict[token]] = term_frequency.get(word_dict[token], 0) + 1
          dtm.append(term_frequency) # dtm에 추가

      return dtm, word_dict

    dtm, word_dict = create_dtm(documents_tokens)

    # 2. TF 계산 (DTM을 NumPy 배열로 변환)
    num_docs = len(documents_tokens)
    num_words = len(word_dict)
    tf_matrix = np.zeros((num_docs, num_words))
    for i, doc_freq in enumerate(dtm):
        for word_id, freq in doc_freq.items():
            tf_matrix[i, word_id] = freq  # 문서 내 단어 빈도 (TF)

    # 3. DF 계산
    df = np.zeros(num_words)  # 각 단어별 문서 빈도
    for word_id in range(num_words):
      for doc_freq in dtm:
        if word_id in doc_freq:
          df[word_id] += 1


    # 4. IDF 계산
    idf = np.log(num_docs / (1 + df))

    # 5. TF-IDF 계산
    tfidf_matrix = tf_matrix * idf  # NumPy broadcasting 활용

    return tfidf_matrix, word_dict

# 영화 데이터 (시놉시스, 장르, 배우)
movies = [
    {
        "synopsis": """
        "나한테 별로 고마워하지 않아도 돼요" 까칠한 어른 윤서
        "한 번 쯤은 자기를 믿어주는 사람이 있으면 좋잖아요" 꿈 없는 청년 수찬

        시청 정기간행물의 인터뷰어 '윤서'에게 사람의 온기는 한여름의 습하고 불쾌한 더위 같은 것.
        그러던 어느 날, 청년 배달원 '수찬'과 실랑이를 벌이고 만다.
        이후 인터뷰 자리에서 우연찮게 다시 만나게 되는데...

        윤서와 수찬, 두 사람의 불편한 만남은 조금씩 서로를 건드린다.
        """,
        "genre": "드라마",
        "actors": ["임선우", "김명찬", "이장유", "박현숙"]
    },
    {
        "synopsis": """
        "선생님, 저랑 사귀실래요?" 적극적인 어른 민주
        "꺼져" 철벽 많은 급식 윤서

        윤서는 학교에서 학생들에게 인기가 매우 많은 선생님이다.
        어느 날, 윤서는 민주로부터 고백을 받게 된다.
        하지만 윤서는 민주를 거절한다.

        윤서와 민주, 두 사람의 아슬아슬한 만남은 계속된다.
        """,
        "genre": "로맨스",
        "actors": ["김민주", "박서준", "이도현"]
    },
    {
       "synopsis": """
        1919년, 3.1 운동 이후 봉오동 전투에서 승리한 독립군의 이야기를 그린 영화.
        """,
        "genre": "액션",
        "actors": ["유해진", "류준열", "조우진"]
    }
]

# 가중치 조합
genre_weights = [1, 3, 5]  # 장르 가중치 후보
actor_weights = [1, 3, 5]  # 배우 가중치 후보


best_similarity = -1  # 가장 낮은 유사도를 -1로 초기화
best_genre_weight = 1
best_actor_weight = 1
best_recommended_movies = []


for genre_weight in genre_weights:
    for actor_weight in actor_weights:
        # 각 영화별 전처리된 토큰 리스트 생성
        documents_tokens = [
            preprocess_synopsis(movie["synopsis"], movie["genre"], movie["actors"], genre_weight, actor_weight)
            for movie in movies
        ]
        # TF-IDF 행렬 생성
        tfidf_matrix, word_dict = create_tfidf_matrix(documents_tokens)

        # 코사인 유사도 행렬 계산 (직접 구현)
        num_movies = len(movies)
        similarity_matrix = np.zeros((num_movies, num_movies))

        for i in range(num_movies):
            for j in range(i + 1, num_movies):
                similarity = cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity  # 대칭 행렬

        # 0번 영화와 다른 영화들 간의 유사도
        similarities = similarity_matrix[0, 1:]  # 0번 영화와 다른 영화(1,2) 간 유사도

        # 현재 가중치 조합에서의 평균 유사도 (0번 영화와 다른 영화 간 유사도의 평균)
        avg_similarity = np.mean(similarities)

        if avg_similarity > best_similarity:
            best_similarity = avg_similarity
            best_genre_weight = genre_weight
            best_actor_weight = actor_weight
            best_recommended_movies = similarities

print(f"가장 좋은 유사도: {best_similarity}")
print(f"최적 장르 가중치: {best_genre_weight}")
print(f"최적 배우 가중치: {best_actor_weight}")

# 추천 영화 인덱스 정렬
recommended_indices = np.argsort(best_recommended_movies)[::-1] # 내림차순 정렬
print(f"추천 영화 (0번 영화 기준): {[movies[i+1]['genre'] for i in recommended_indices]}") # 인덱스 1, 2에 해당하는 영화 제목 출력
print(best_recommended_movies)



**주요 변경 사항:**

1.  **`preprocess_synopsis` 함수:**
    *   `genre_weight`, `actor_weight` 매개변수를 추가하여 장르와 배우 정보를 몇 번 반복해서 토큰 리스트에 추가할지 결정합니다.
    *   `okt.nouns(synopsis)`: KoNLPy의 `Okt` 형태소 분석기를 사용하여 명사만 추출합니다.
    *   불용어 제거를 명사에도 적용합니다.
    *   장르와 배우 정보를 소문자로 변환하여 추가합니다.

2.  **`create_tfidf_matrix` 함수:**
    *   TfidfVectorizer를 사용하지 않고 직접 구현합니다.

3.  **메인 로직:**
    *   여러 `genre_weight`와 `actor_weight` 조합을 시도하면서 가장 좋은 유사도 점수를 보이는 조합을 찾습니다.
    *   `cosine_similarity` (직접 구현)를 사용하여 유사도 행렬 계산.
    *   0번 영화와 다른 영화들 간의 유사도를 기준으로 가장 좋은 가중치를 찾고, 해당 가중치에서의 추천 영화를 출력합니다.

**실행 결과 및 해석:**

*   코드를 실행하면 다양한 가중치 조합에 따른 유사도 점수와 추천 영화 목록이 출력됩니다.
*   가장 좋은 유사도 점수는 0번 영화와 다른 영화들 간의 평균 코사인 유사도를 기준으로 합니다. (값이 높을수록 좋음).
*   최적의 가중치는 해당 점수를 최대화하는 장르 가중치와 배우 가중치입니다.
*  `recommended_indices`는 유사도가 높은 순으로 정렬된 인덱스를 나타냅니다.

**가중치 튜닝:**

*   현재 코드는 간단한 그리드 탐색(grid search) 방식으로 가중치를 튜닝합니다.
*   더 정교한 튜닝을 위해서는 교차 검증(cross-validation)과 함께 다른 최적화 알고리즘(예: 베이지안 최적화)을 사용할 수 있습니다.

**참고:**

*   실제 영화 추천 시스템에서는 훨씬 더 많은 영화 데이터를 사용하고, 사용자-영화 상호작용 데이터(예: 평점)를 함께 고려하는 협업 필터링(collaborative filtering) 등의 고급 기법을 사용합니다.
*   이 코드는 콘텐츠 기반 추천(content-based filtering)의 기본적인 예시를 보여줍니다.


## KoNLPy (코엔엘파이)

**KoNLPy**는 한국어 정보 처리를 위한 파이썬 패키지입니다. 형태소 분석, 품사 태깅, 구문 분석 등 다양한 자연어 처리 기능을 제공합니다. 특히, 한국어는 영어와 달리 형태소(의미를 가지는 가장 작은 단위) 분석이 중요하기 때문에 KoNLPy는 한국어 텍스트 분석에 필수적인 도구입니다.

**주요 기능:**

*   **형태소 분석기:** KoNLPy는 여러 가지 형태소 분석기를 내장하고 있습니다.
    *   **Okt (Open Korean Text, 구 Twitter):** 속도가 빠르고, 신조어나 비표준어 처리 능력이 좋습니다.
    *   **Kkma (꼬꼬마):** 분석 품질이 좋지만, 속도가 상대적으로 느립니다.
    *   **Komoran (코모란):** 오탈자에 강하고, 사용자 사전 추가 기능이 있습니다.
    *   **Hannanum (한나눔):** KAIST에서 개발한 형태소 분석기입니다.
    *   **Mecab (메카브):** 일본어 형태소 분석기를 한국어에 맞게 수정한 것으로, 속도가 매우 빠릅니다. (별도 설치 필요)
*   **품사 태깅 (POS Tagging):** 각 형태소에 품사를 태깅합니다. (예: 명사, 동사, 형용사 등)
*   **명사 추출:** 텍스트에서 명사만 추출합니다.
*   **구문 분석:** 문장의 구조를 분석합니다. (KoNLPy에서 직접 지원하지는 않지만, 다른 라이브러리와 함께 사용 가능)
*   **말뭉치 (Corpus):** KoNLPy는 한국어 말뭉치(텍스트 데이터)에 접근할 수 있는 기능을 제공합니다. (예: 대한민국 헌법, 국회 의사록 등)

**설치:**

```bash
pip install konlpy
```

**간단한 사용 예시 (Okt):**

```python
from konlpy.tag import Okt

okt = Okt()

text = "한국어 자연어 처리는 재미있습니다."

# 형태소 분석
morphs = okt.morphs(text)
print("형태소:", morphs)

# 품사 태깅
pos = okt.pos(text)
print("품사 태깅:", pos)

# 명사 추출
nouns = okt.nouns(text)
print("명사:", nouns)
```

**출력:**

```
형태소: ['한국어', '자연어', '처리', '는', '재미있습니다', '.']
품사 태깅: [('한국어', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재미있습니다', 'Adjective'), ('.', 'Punctuation')]
명사: ['한국어', '자연어', '처리']
```



## TfidfVectorizer

**TfidfVectorizer**는 텍스트 데이터를 TF-IDF (Term Frequency-Inverse Document Frequency) 값으로 변환하는 sklearn(사이킷런)의 클래스입니다. TF-IDF는 텍스트 마이닝에서 단어의 중요도를 나타내는 데 사용되는 가중치입니다.

**TF-IDF:**

*   **TF (Term Frequency):** 단어가 문서 내에서 얼마나 자주 나타나는지를 나타냅니다.
*   **IDF (Inverse Document Frequency):** 단어가 여러 문서에서 얼마나 드물게 나타나는지를 나타냅니다.
*   **TF-IDF:** TF와 IDF를 곱한 값으로, 특정 문서에서 자주 등장하고 다른 문서에서는 드물게 등장하는 단어에 높은 가중치를 부여합니다.

**TfidfVectorizer의 주요 기능:**

*   **텍스트 전처리:**
    *   소문자 변환
    *   토큰화 (단어 분리) - 기본적으로 공백 기준
    *   불용어(stop words) 제거 (영어 불용어 기본 제공, 사용자 정의 가능)
*   **TF-IDF 계산:** 각 문서의 각 단어에 대한 TF-IDF 값을 계산하여 행렬 형태로 반환합니다.
*   **희소 행렬(sparse matrix):**  결과 TF-IDF 행렬은 대부분의 값이 0인 희소 행렬로 표현됩니다. (메모리 효율적)
*   **다양한 매개변수:**
    *   `max_df`:  DF(document frequency)가 지정된 임계값보다 높은 단어는 무시합니다. (너무 자주 나타나는 단어 제거)
    *   `min_df`: DF가 지정된 임계값보다 낮은 단어는 무시합니다. (너무 드물게 나타나는 단어 제거)
    *   `max_features`: 최대 단어 수를 제한합니다.
    *   `ngram_range`:  n-gram 범위를 지정합니다. (예: (1, 2)는 unigram과 bigram을 모두 사용)
    *   `stop_words`: 불용어 리스트를 지정합니다. ('english' 또는 사용자 정의 리스트)
    *   `tokenizer`: 사용자 정의 토큰화 함수를 지정합니다.
    *   `norm`: 정규화 방법 ('l1', 'l2', None)을 지정합니다.

**설치 (sklearn):**

```bash
pip install scikit-learn
```

**간단한 사용 예시:**

```python
from sklearn.feature_extraction.text import TfidfVectorizer

# 문서 집합
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer()

# TF-IDF 행렬 계산
tfidf_matrix = vectorizer.fit_transform(documents)

# 단어 사전 (feature names)
feature_names = vectorizer.get_feature_names_out()
print("단어 사전:", feature_names)

# TF-IDF 행렬 (희소 행렬)
print("TF-IDF 행렬:\n", tfidf_matrix)  # 희소 행렬 표현
print("TF-IDF 행렬 (밀집 행렬):\n", tfidf_matrix.toarray()) # 밀집 행렬로 변환 (보기 편함)

```

**출력 (일부):**

```
단어 사전: ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
TF-IDF 행렬:
   (0, 7)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 3)	0.38408524091481483
  (0, 1)	0.46979138557992045
  (0, 6)	0.281088674033753
  ...
TF-IDF 행렬 (밀집 행렬):
 [[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.28108867 0.46979139 0.        ]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.        ]
 ...
```

**설명:**

*   `vectorizer.fit_transform(documents)`:
    *   `fit()`: 문서 집합(`documents`)으로부터 단어 사전을 만들고 IDF를 계산합니다.
    *   `transform()`: 각 문서를 TF-IDF 벡터로 변환합니다.
    *   `fit_transform()`: `fit()`과 `transform()`을 한 번에 수행합니다.
*   `vectorizer.get_feature_names_out()`: 단어 사전(vocabulary)에 있는 단어들을 가져옵니다.
*   `tfidf_matrix`: TF-IDF 행렬 (희소 행렬 형태)
*   `tfidf_matrix.toarray()`: 희소 행렬을 밀집 행렬(dense matrix)로 변환 (보기 쉽게)

**KoNLPy와 TfidfVectorizer 함께 사용:**

KoNLPy의 형태소 분석기를 TfidfVectorizer의 `tokenizer` 매개변수에 지정하여 한국어 텍스트를 처리할 수 있습니다.

```python
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()

def tokenizer(text):
    return okt.nouns(text)  # 명사만 추출

# TfidfVectorizer 객체 생성 (tokenizer 지정)
vectorizer = TfidfVectorizer(tokenizer=tokenizer)

# ... (TF-IDF 행렬 계산 등) ...
```
